1.
Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [1]:
import pandas as pd

In [2]:
water = pd.read_csv('water.txt', sep = '\t')
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
from scipy.stats import pearsonr
corr_p, p_val = pearsonr(water.hardness, water.mortality)
print(round(corr_p, 4))

-0.6548


2.
В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [4]:
from scipy.stats import spearmanr
corr_s, p_val = spearmanr(water.hardness, water.mortality)
print(round(corr_s, 4))

-0.6317


3.
Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [7]:
corr_p_south, p_val = pearsonr(water[water.location == 'South'].hardness, water[water.location == 'South'].mortality)
corr_p_north, p_val = pearsonr(water[water.location == 'North'].hardness, water[water.location == 'North'].mortality)
print(round(corr_p_south, 4), round(corr_p_north, 4))

-0.6022 -0.3686


Answer = -0.3686

4.
Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [9]:
N = 203 + 239 + 718 + 515
S = (203 + 718) / N
P = (203 + 239) / N
corr_m = (203/N - S*P)/(P*S*(1 - S)*(1 - P))**0.5
print(round(corr_m, 4))

-0.109


5.
В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$ , нужно ввести 8).

In [11]:
from scipy.stats import chi2_contingency

In [12]:
chi2_contingency([[203, 718], [239, 515]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[243.03402985, 677.96597015],
        [198.96597015, 555.03402985]]))

Answer = 5

6.
В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [23]:
from scipy import stats
import numpy as np

In [28]:
men = [1] * 239 + [0] * 515
women = [1] * 203 + [0] * 718
len(men), len(women)

(754, 921)

In [24]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [29]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(men, women))

#proportion_confint([239 - 203, 515 - 718], 2)

95% confidence interval for a difference between proportions: [0.053905, 0.139222]


Answer = 0.0539

7.
Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [33]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [35]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(men, women)))

p-value: 0.000008


Answer = 6

8.
Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

Не доволен	Более или менее	Доволен

Не очень счастлив	197	111	33

Достаточно счастлив	382	685	331

Очень счастлив	110	342	333

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [38]:
arr = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

In [48]:
arr

array([[197, 111,  33],
       [382, 685, 331],
       [110, 342, 333]])

In [50]:
chi2, _, _, _ = chi2_contingency(arr)
chi2

293.68311039689746

Answer = 293.6831

9.
На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [43]:
_, p_val, _, _ = chi2_contingency(arr)
p_val

2.4964299580093467e-62

Answer = 62

10.
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

$$ \phi _{c} (X_{1}^{n}, X_{1}^{n}) = \sqrt{\frac{\chi ^{2}(X_{1}^{n}, X_{1}^{n})}{n(min(K_{1},K_{2})- 1)}} $$

In [62]:
coef_vkr = ((chi2**2/(np.sum(arr)*(len(arr)-1)))) ** 0.5
coef_vkr

4.133512906544862

Вроде по формуле считаю коэффициент Крамера, но никак не получается ответ в промежутке от 0 до 1 и, соответственно, это ответ не правильный.